In [41]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from ipywidgets import HBox, VBox,Layout
import numpy as np
import random
import uuid
import datetime
import geopandas as gpd
import matplotlib.pyplot as plt; plt.rcdefaults()
import copy
import pyproj
pyproj.datadir.set_data_dir('/Users/shai/anaconda3/envs/geo_env/share/proj')

pd.options.display.max_columns=50

In [42]:
#!pip install openpyxl

# Classes

## Random 🎲🎲 
# (Used only in Buy In Buy Out might be not necessary)

In [43]:
class ridt_random:
    
    def random_dates_list(start, end, n):
        """ recieves a START date
        and an END date
        and a N number of random Dates"""
        start_year = start.year
        end_year = end.year
        random_year = random.randint(start_year, end_year)
        return pd.to_datetime(str(random_year)+'-5-1')

## RIDT_Properites

In [105]:
class ridt_properties:
    # default constructor
    def __init__(self, js_bldgs):
        """Recieves json building file and returns a property dataset
        js_bldgs: json_bldgs
        """
        self.json_bldgs = js_bldgs
        self.json_bldgs_slim = js_bldgs[['Bldg Proje', 'bld_key', 'cnstrct_yr', 'Total Unit', 'Category', 'Group', 'geometry']]
        self.bldgs_list = js_bldgs['Bldg Proje'].unique() # list of bldgs
        self.bldgs_ids = js_bldgs['bldgs_id'].unique() #List of bldgs IDs
        ridt_widgets.select_bldg = widgets.Dropdown(
            #add building list to widgets
            options=self.bldgs_list,
            value="Rivercross",
            description='Bldgs:'
        )
        
        self.bldg_dict = self.json_bldgs[['bldgs_id', 'cnstrct_yr', 'Bldg Proje', 'Category', 'Group']] #similar to bldgs_slim
        self.ds = self.iterate_json_bldgs()
    
    def iterate_json_bldgs(self):
        """Iterates through each json_bldg index and creates property dataset"""
        count = 0
        for i in (self.json_bldgs.index):
            cb = self.json_bldgs.loc[i]  # current building
            cb_ = ridt_bldg_obj(cb)
            Ap_datasets = ridt_bldgs.generate_properties_in_bldg(cb_)
            if (cb_.group == 'NorthTown'):
                Ap_datasets = ridt_SouthTown.generateMarket(cb_, Ap_datasets,0)
            elif (cb_.group == 'SouthTown'):
                Ap_datasets = ridt_SouthTown.generateMarket(cb_, Ap_datasets,0)
            elif (cb_.group == 'WIRE'):
                Ap_datasets = ridt_WIRE.updateRoomsInWire(cb_, Ap_datasets)
                Ap_datasets = ridt_WIRE.set_market(cb_, Ap_datasets)
            else:
                Ap_datasets = ridt_bldgs.afford_price_Other(cb_, Ap_datasets)

            if (count == 0):  # First iteration
                property_df = Ap_datasets
                count += 1
            else:
                frames = [property_df, Ap_datasets]
                property_df = pd.concat(frames)
        property_df.reset_index(inplace=True, drop=True)
        return property_df
        
    @staticmethod
    def get_avilable_index_units(remove_agents, bld_ts,bld_name):
      """Get Avilable Index when given departed agents and bld name and bld dataset"""
      tu = bld_ts.ds[bld_ts.ds['Bldg Proje']==bld_name].iloc[0]['Total Unit']
      ap_index = np.array([ i for i in range(tu)])
      avilable_index = np.delete(ap_index, remove_agents)
      return avilable_index

## RIDT Info

In [45]:
class ridt_info:
    affordable_models_southtown = {
    'Southtown':{
        'type':'multi',
        'a' :{
            'ap_percent': 0.15,
            'ami_percent_min':0.5,
            'ami_percent_max':0.8            
            },
        'b':{
            'ap_percent': 0.35,
            'ami_percent_min':1.2,
            'ami_percent_max':1.48            
            }
        }
    }
    affordable_models_not_southtown = {
        'Other' : {
            'type':'default',
            'Upper':{
                'ami_percent_min':1.2,
                'ami_percent_max':1.48                
            },
            'Middle':{
                'ami_percent_min':0.8,
                'ami_percent_max':1.2                 
            },
            'LowMed':{
                'ami_percent_min':0.1,
                'ami_percent_max':0.3                  
            }
        },
        'Northtown' : {
            'type':'default',
            'Upper':{
                'ami_percent_min':1.2,
                'ami_percent_max':1.48                
            },
            'Middle':{
                'ami_percent_min':0.8,
                'ami_percent_max':1.2                 
            },
            'LowMed':{
                'ami_percent_min':0.1,
                'ami_percent_max':0.3                  
            }
        },
        'Student':{
            'type':'default',
            'Middle':{
                'ami_percent_min':0.8,
                'ami_percent_max':1.2                 
            }

        }
    }
    """Provides General Information for simulation"""
    def non_affordable_approximation(year_):
        """Provide an approximation of market income based on year that creates an AMI, needs to be calibrated by percentage of ami"""
        ami_ap = ridt_info.simulate_ami_using_polynom(year_)
        non_affordable ={
            'min': ami_ap*0.85,
            'max': ami_ap*2
            }
        return non_affordable

    def simulate_ami_using_polynom(year_):
        """returns a simulated ami for calculating the peoples income who come to live"""
        #This is based on Average mean Income and NYC AMI report (no graph provided)
        #https://www.notion.so/sociotechtechnion/Calibration-Income-6937fc22d354413d9e787d9e68651193
        x_ = year_-1900 #
        ami_ = 9.8125*np.power(x_,2)-844.375*x_+51025 
        return ami_
    
    @staticmethod
    def get_Income_based_on_marketRent_and_size_year(sqft,year_):
      """
      The Function returns a list of minimum income requiered and max income estimated
      based on Size of appartment and a given year
      """
      ppf = ridt_info.rent_per_feet_market(year_)
      minmum_Income = ppf*sqft*40
      non_affordable ={
          ### Change according to Year!!!
          'min': minmum_Income*1.0,
          'max': minmum_Income*2
          }
      return pd.Series(non_affordable)

    @staticmethod
    def rent_per_feet_market(year_):
      """Function that simulates the increase in price rent
      based on given year
      https://docs.google.com/spreadsheets/d/1znc9L9kiLTPZoeeP5kaL-9WtPn5IKFvH/edit?usp=drive_web&ouid=106407473734356401683&rtpof=true
      
      """
      a = 0.0702
      b = - 138.29
      return round(a*year_+b,2)


## Buildings Object

In [46]:
class ridt_bldgs:
    def generate_properties_in_bldg(cb1_): 
        """The Function recieves the current building 
        and returns the properties and it's properties"""
        total_units = cb1_.total_units #Total number of Agents in the building
        bld_key = [cb1_.bldg_key]*total_units #Defines the number of the building
        property_ds = {'ap_index' : [i for i in range(total_units)],
                      'bld_key' : bld_key} # define the dictionary for the dataset
        Ap_Datasets = pd.DataFrame.from_dict(property_ds)
        #The building key and the door number
        Ap_Datasets['address_door']=Ap_Datasets['bld_key']+"_"+Ap_Datasets['ap_index'].astype(str)
        #Building Name
        Ap_Datasets['Building Name'] = cb1_.name_
        #Construction year
        Ap_Datasets['built_year']= cb1_.constr_date
        #is the apartment orginaly affordable?
        Ap_Datasets['affordable_living'] = 0
        # by default all the apartments are not for elderly
        Ap_Datasets['elderly_lving'] = 0
        #Building Group Category
        Ap_Datasets['Group'] = cb1_.group
        #by default apartments are non rnetal
        Ap_Datasets['Rental_Units'] = 0
        Ap_Datasets['rentalType'] = ''
        #by default apartments are non rental
        Ap_Datasets['ApartmentType'] = 'Other'
        Ap_Datasets['annual_utilities'] = 3000
        if cb1_.rental_units>0:
            Ap_Datasets.loc[0:cb1_.rental_units-1,'Rental_Units'] = 1
            Ap_Datasets.loc[0:cb1_.rental_units-1,'ApartmentType'] = 'Rental'
        Ap_Datasets['Ap_sq_feet_size'] = 1100 # default
        Ap_Datasets['Ap_number_of_rooms'] = 2 # default
        if cb1_.protected=='1':
            Ap_Datasets['protected']=True
        else:
            Ap_Datasets['protected']=False
        ela = cb1_.elderly #elderly living agents
        if ela>0:
            # Check if it's wire if not all other cases
            Ap_Datasets.loc[0:ela,'elderly_lving'] = 1
        if cb1_.affordable_units>0:
            Ap_Datasets.loc[0:cb1_.affordable_units,'affordable_living'] = 1
        return Ap_Datasets

    @staticmethod
    def afford_price_Other(cb,App_datasets):
        """generates income range min and max for Non WIRE
        in addition it generates Upper Income
        cb: current building
        App_datasets: property dataset in the current building"""
         #calculate affordability according to the building
        year_ = cb.constr_date.year
        affordable_units = cb.affordable_units
        non_affordable = ridt_info.non_affordable_approximation(year_)
        if affordable_units>0:
            cat = cb.affordable_Cat
            type_ = ridt_info.affordable_models_not_southtown[cat]['type']
            a_type = cb.affordable_sub
            income_min,income_max = ridt_agent.helper_function_2_cal_income(a_type,cat,year_)
            App_datasets.loc[0:affordable_units-1,'Low_income'] = income_min
            App_datasets.loc[0:affordable_units-1,'Upper_income'] = income_max
            #Upper Income Needs 
            App_datasets.loc[affordable_units :,'Low_income'] = non_affordable['min']
            App_datasets.loc[affordable_units :,'Upper_income'] = non_affordable['max']

        else:
            """for a given building the function claculates the income"""
            App_datasets['Low_income'] = non_affordable['min']
            App_datasets['Upper_income'] = non_affordable['max']
        return App_datasets

    @staticmethod
    def get_avilable_index_units(remove_agents, bld_ts,bld_name):
      """Get Avilable Index when given departed agents and bld name and bld dataset"""
      tu = bld_ts.ds[bld_ts.ds['Bldg Proje']==bld_name].iloc[0]['Total Unit']
      ap_index = np.array([ i for i in range(tu)])
      avilable_index = np.delete(ap_index, remove_agents)
      return avilable_index

## Building Object

In [47]:
class ridt_bldg_obj:
    def __init__(self,cb):
        """The object recieves the json row and returns an object of a specfic building with it's info"""
        self.name_ = cb['Bldg Proje']
        self.bldg_key = cb['bldgs_id']
        self.constr_date = pd.to_datetime(cb['cnstrct_yr'].astype(str)+"-1-"+"1")
        self.constr_year = cb['cnstrct_yr']
        self.heightroof = cb['heightroof']
        self.address = cb['Address']
        self.total_units = cb['Total Unit']
        self.affordable_units = cb['Afford_uni'].astype(int)
        self.elderly = cb['elderly_affordable']
        self.affordable =cb['non_elderly_affordable']
        self.affordable_Cat = cb['affordable_Cat']
        self.affordable_sub = cb['affordable_sub']
        self.rental_units = cb['Rental_Units']
        self.protected = cb['protected']
        self.geometry = cb['geometry']
        self.group = cb['Group']
        self.category = cb['Category']

## Buildings Time Series

In [48]:
class ridt_buildings_ts_obj:
    def __init__(self, bldg_ds,b_events):
        """recieves a building dataset and convert it to a time series based on an event external file"""
        self.ds=''
        self.build_base_ds(bldg_ds)
        self.add_events_to_ds(b_events)
            
    def build_base_ds(self,bldg_ds):
        """builds a slim version of the building dataset (with no geometry)"""
        self.ds = bldg_ds[['Bldg Proje',
 'cnstrct_yr',
 'Total Unit',
 'Non_Afford',
 'Afford_uni',
 'Category',
 'bldgs_id',
 'affordability_percent',
 'elderly_affordable',
 'non_elderly_affordable',
 'bld_key',
 'afford_percent',
 'affordable_Cat',
 'affordable_sub',
 'Rental_Percent',
 'Rental_Units',
 'protected']].copy()
        self.ds['Event'] ="No Events"
        self.ds['start_event'] = pd.to_datetime(self.ds['cnstrct_yr'].astype(str)+"-01-01")
        self.ds['end_event'] = ridt_agents_ts_obj.last_date
        self.ds['event_counter'] =0
        self.ds['event comment'] =''
    
    def add_events_to_ds(self,b_events):
        """adds building events from external excel file"""
        #add i loop
        for i in range(len(b_events.index)):
          #Rivercross Pilot!
          cbe = b_events.loc[i] #current building event
          selected_b = self.ds.loc[(self.ds['Bldg Proje']==cbe['building_Project'])].copy()
          selected_b = selected_b.loc[selected_b.index[-1]]
          #print(selected_b)
          selected_b['Event'] = cbe['event']
          selected_b['start_event'] = cbe['start_date']
          selected_b['end_event'] = cbe['end_date']
          selected_b['event_counter'] = selected_b['event_counter']+ 1
          if (cbe['event']=='Market Rate'):
            col_list = ['Afford_uni','affordability_percent','elderly_affordable','non_elderly_affordable''Rental_Percent','Rental_Units','protected','afford_percent']
            for col in col_list:
              selected_b[col] = 0
            selected_b['Non_Afford']  = selected_b['Total Unit']

          selected_b = pd.DataFrame(selected_b).transpose()
          self.ds = pd.concat([self.ds,selected_b])
          self.ds.reset_index(inplace=True,drop=True)
      
    

## WIRE

In [97]:
wire_bldg_data

,bld_name,Income Category,Bedroom,Elderly,Lower,Upper
0,island house,Middle,0,No,"$ 33,700.36","$ 50,550.54"
1,NaN,NaN,1,No,"$ 46,851.72","$ 70,688.56"
2,NaN,NaN,2,No,"$ 60,414.06","$ 90,826.58"
3,NaN,NaN,3,No,"$ 80,552.08","$ 140,966.14"
4,NaN,NaN,4,No,"$ 94,114.42","$ 164,802.98"
5,Rivercross,Upper,1,No,"$ 53,838.38","$ 80,552.08"
6,NaN,NaN,2,No,"$ 69,866.60","$ 113,019.50"
7,NaN,NaN,3,No,"$ 92,881.48","$ 169,323.76"
8,Roosevelt Landings,Low,0,No,"$ 13,562.34","$ 20,959.98"
9,NaN,NaN,0,Yes,"$ 4,109.80","$ 11,918.42"


In [93]:
class ridt_WIRE:
    wire_projects = pd.Series(['Roosevelt Landings','Rivercross','island house','Westview'])
    wire_dict = {'island house': '543_585',
     'Roosevelt Landings': '500_580',
     'Rivercross': '501_541',
     'Westview': '587_625'}
    
    @staticmethod
    def updateRoomsInWire(cb,App_datasets):
        """Update Number of Rooms in WIRE"""
        bldg_name = cb.name_
        print('bld name',bldg_name)
        wbd_subset = wire_bldg_data[wire_bldg_data['bld_name']==bldg_name].reset_index(drop=True) #Wire Buidling Data Subset
        print('wbd_subset')
        f_c=0 #floor_count
        for j in wbd_subset.index:
            wbds_room_subset = wbd_subset.loc[j]
            elderly_flag=0
            if (wbds_room_subset['Elderly']==1):
                elderly_flag=1
            r_c=wbds_room_subset['Units']+f_c #roof_count
            App_datasets.loc[f_c:r_c-1,'Ap_number_of_rooms'] =wbds_room_subset['Bedroom'] 
            App_datasets.loc[f_c:r_c-1,'Ap_sq_feet_size'] =wbds_room_subset['sq_feet'] 
            f_c = r_c
        return App_datasets
      
  
    @staticmethod
    def set_market(cb_, App_datasets):
      """"""
      bldg_name = cb_.name_
      #distrivute agents to rooms and prices
      wbd_subset = wire_bldg_data[wire_bldg_data['bld_name']==bldg_name].reset_index(drop=True) #Wire Buidling Data Subset
      f_c=0 #floor_count
      for j in wbd_subset.index:
          wbds_room_subset = wbd_subset.loc[j]
          low_t = wbds_room_subset['Low_today']
          up_t = wbds_room_subset['Upper_today']
          r_c=wbds_room_subset['Units']+f_c #roof_count
          App_datasets.loc[f_c:r_c-1,'Low_income'] = low_t
          App_datasets.loc[f_c:r_c-1,'Upper_income'] = up_t
          f_c = r_c
      return App_datasets

    @staticmethod
    def update_Wire_To_Market_rate(p_ds, bldg_events, dfts):
        """This function updates the Populatino Time Series according to the WIRE Scenario
        Input
        -----
        p_ds: building object (information about building)
        bldg_events: the events of the building
        dfts: Population TimeSeries
        Output
        ------
        the simulation returns a new dataset with the agents modification according to WIRE Scenario"""
        bld_ts = ridt_buildings_ts_obj(
            p_ds.json_bldgs, bldg_events)  # Adds time series to the building DS
        # Will need to update for non WIRE eventually
        abe = bld_ts.ds[bld_ts.ds['Event'] != "No Events"]
        bld_list = list(abe['Bldg Proje'].unique())
        dfts = ridt_WIRE.updtate_write_to_market_rate_iterate_bld_list(
            bld_list, abe, dfts, bld_ts, p_ds)
        dup = dfts.ds[['agentID','tenant_cycle']]
        dup['i']=dup.index
        keep = dup.groupby('agentID').agg({'i':'first'}).reset_index()['i']
        dfts.ds = dfts.ds[dfts.ds.index.isin(keep)]
        return(dfts)

    @staticmethod
    def updtate_write_to_market_rate_iterate_bld_list(bld_list, abe, dfts, bld_ts,p_ds):
        for cb in bld_list:
            print('bulding', cb)
            print('')
            fe = abe.loc[(abe['Bldg Proje'] == cb) & (abe['Event'] == 'Flaged')].squeeze()  # fe flaged Event
            rc = ridt_agents_ts_obj(
                dfts.filter_ds_by_building(fe['Bldg Proje']).ds)
            if len(rc.ds) > 0:
                rc.ds = ridt_WIRE.buy_out_specific_building (rc.ds,fe)
                rc.ds = ridt_WIRE.buy_in_specific_building (rc.ds,fe)                    
                # Remove from the main dataset Rivercross
                dfts.drop(rc.ds, "Building Name")
                pem = abe.loc[(abe['Bldg Proje'] == cb) & (abe['Event'] == 'Market Rate')].squeeze()  # me market event Market rate pilot for Rivercros
                rc.update_cleanup_agents('Rental')
                se = pem.start_event  # start event
                ee = pem.end_event
                b_ds = rc.copy()  # switching to procedures

                b_ds = ridt_WIRE.iterate_years_WIRE_update_market(se, ee, b_ds, bld_ts,cb,p_ds,'Market Buy')
                dfts.concat([b_ds.ds])
            else:
                continue
        return dfts

    @staticmethod
    def update_market_rate_current_year(year_,last_year,building_ds,bld_ts,bld_name,p_ds,comment):
      """This function is part of a bigger loop function that iterates throughout the Market Rate Period of the building
      year_: that is the ye iterator year in the list
      last_year: that is the last year to check
      building_ds: that is the building dataset reference to update"""
      ye_ = str(year_) #turn to string
      se_ = pd.to_datetime(ye_+'-1-1') # start event
      middle_of_year = pd.to_datetime(ye_+'-5-1')
      if (year_<last_year):
          #current_year_agents
          #remove_dead_agents
          #remove_rent_burden
          #remove_total_burden
          #generate new agents
          #add history data
          #append

          da2 = building_ds.find_agents_to_remove_market_rate_for_year(se_) #parameters used in False
      else:
          ag_sub = building_ds.filter_ds_by_full_year_date(se_)
          protected = ag_sub.ds[ag_sub.ds['protected']==True]
          protected['cause'] ='End of contract'
          ag_sub.ds = ag_sub.ds[~ag_sub.ds['agentID'].isin(protected['agentID'])]
          #protected = building_ds.remove_protected_for_year(se_)
          ey_ = pd.to_datetime(str(se_.year+1)+"-1-1")
          dead_agents = ag_sub.ds[(ag_sub.ds['death_date']>=se_) &(ag_sub.ds['death_date']<ey_)]
          dead_agents['cause'] ='death'
          da2 = pd.concat([protected,dead_agents]).drop_duplicates()
          da2['move_out'] = middle_of_year
      

      print('Agents to remove:', len(da2))
      if (len(da2)>0):
        da2.drop_duplicates(inplace=True)
        building_ds.drop_future_agent_cycles(da2)
        #da2 = ridt_agents_ts_obj.update_status_of_displaced_agents_market_rate(da2,se_)  
        current_year =ridt_agents_ts_obj(building_ds.filter_ds_by_full_year_date(se_).ds)
        #building_ds.drop(current_year.ds,'agentID') #drop displaced Agents
        building_ds.ds = building_ds.ds.loc[~building_ds.ds.index.isin(current_year.ds.index)]
        ai = ridt_WIRE.WIRE_get_index_of_apartments_to_inhabit_market_rate_for_year(current_year.ds,da2,bld_ts,bld_name,se_)
        print('Vacant Aparments', len(ai))
          
        new_agents = ridt_agents_ts_obj(ridt_WIRE.create_and_calibarte_new_agents_market_rate_WIRE(p_ds,bld_name,ai,se_,comment,da2))
        new_agents.ds['annual_return_mortgage'] = ridt_agent_economy.update_mortgage_for_year(new_agents.ds,year_,True)
        
        new_agents.ds['income'] = (new_agents.ds['annual_return_mortgage']/0.3)
        new_agents.ds['income'] = (new_agents.ds['income'].apply(lambda x: x+ random.randint(50000,100000))).astype(int) # Assumption
        new_agents.ds['annual_utilities']=3000
        new_agents.update_cleanup_agents('Other')
        print('Agents Created', len(new_agents.ds))
        if (len(new_agents.ds)>len(ai)):
            print('More Agents created than vacant!')
        current_year.drop(da2,'agentID')
        current_year.concat([da2,new_agents.ds]) #Special Concat
        building_ds.concat([current_year.ds])
        building_ds.ds.reset_index(inplace=True,drop=True)
      return building_ds
        
    @staticmethod
    def create_and_calibarte_new_agents_market_rate_WIRE(p_ds,bld_name,ai,start_year,comment,Agents_to_displace):
      na = ridt_demographic.generate_new_marketRate_agents_from_avilable_ap(p_ds,bld_name,ai,start_year,comment)
      col_to_remove =['address_door','tenant_cycle','agentID','cause']
      ad_slim = Agents_to_displace[col_to_remove]
      col_to_remove_x = [col + "_x" for col in col_to_remove]
      na_2 = pd.merge(na,ad_slim, how='left',on='address_door',suffixes=('','_x'))
      na_2['tenant_cycle'] = (na_2['tenant_cycle_x']+1).fillna(0)
      na_2['previous_agentID'] = (na_2['agentID_x']).fillna('')
      na_2.drop(columns=col_to_remove_x[1:],inplace=True)
      return (na_2)

    @staticmethod
    def WIRE_get_index_of_apartments_to_inhabit_market_rate_for_year(ds,ad,bld_ts,bld_name,start_year):
      """"ad displaced agents"""
      end_of_year = pd.to_datetime(str(start_year.year+1)+"-1-1")
      StayingAgents = ds[~ds['agentID'].isin(ad['agentID'])]['ap_index'].copy()
      ai = ridt_bldgs.get_avilable_index_units(StayingAgents,bld_ts,bld_name)
      return (ai)
    
    @staticmethod
    def iterate_years_WIRE_update_market(se,ee,building_ds,bld_ts,bld_name,p_ds,comment):
      """"""
      for ye in range(se.year,ee.year+1):
        print('year:',ye)
        if len(building_ds.ds)>0:
            building_ds = ridt_WIRE.update_market_rate_current_year(ye,ee.year,building_ds,bld_ts,bld_name,p_ds,comment)
      return building_ds
    
 
    @staticmethod
    def buy_out_specific_building(ds, ce):
        """updates data set, choose 1% of tenants who buy out
        the function also calls for future cycle tenats and removes them.
        sd: specific buidling"""
        con1 = ds['Building Name'] == ce['Bldg Proje']
        # active agents start action needs to be before start date and end action after end date
        con2 = ((ds['move_in'] <= ce['start_event']) &
        (ds['move_out'] >= ce['end_event'])) & (ds['elderly_lving'] == 0) & (ds['comment 1'] == '')
        # select agents from selected building
        safsb = ds[con1 & con2].copy()
        # sample 1% of the agents and save a copy of them (those will buy out!!!)
        safsb_buy_out = safsb.sample(frac=0.01)  # 1% buy out

        safsb_buy_out['cause'] = 'Buy Out'
        safsb_buy_out['comment 1'] = '55000$'
        safsb_buy_out['comment 2'] = 'Buy Out'
        safsb_buy_out['move_out'] = ridt_random.random_dates_list(
            ce['start_event'], ce['end_event'], len(safsb_buy_out.index))
        ds.loc[safsb_buy_out.index] = safsb_buy_out
        ds.reset_index(inplace=True, drop=True)
        return(ds)

    @staticmethod
    def buy_in_specific_building(ds, ce):
        """updates data set, choose 1# of tenants who buy out
        the function also calls for future cycle tenats and removes them.
        sd: specific buidling"""
        con1 = ds['Building Name'] == ce['Bldg Proje']
        ds[con1]
        # active agents start action needs to be before start date and end action after end date
        con2 = ((ds['move_in'] <= ce['start_event']) & 
        (ds['move_out'] >= ce['end_event'])) & ((ds['elderly_lving'] ==0) & (ds['comment 1']==''))
        # select agents from selected building
        safsb = ds[con1 & con2].copy()
        # sample 1% of the agents and save a copy of them (those will buy out!!!)
        safsb_buy_in = safsb.sample(frac=0.03)  # 1% buy out
        # !!!!! NEEED TO DECIDE WHAT HAPPENDS to FUTURE CYCLE after Death
        safsb_buy_in['cause'] = 'Buy IN'
        safsb_buy_in['comment 1'] = '55000$'
        safsb_buy_in['comment 2'] = 'Buy IN'
        safsb_buy_in['move_out'] = ridt_random.random_dates_list(ce['start_event'], ce['end_event'], len(safsb_buy_in.index))
        ds.loc[safsb_buy_in.index] = safsb_buy_in
        # add the same agent now as a BUYER
        safsb_buy_in = safsb_buy_in.copy()
        safsb_buy_in['cause'] = 'death'
        # safsb_buy_in['cause']='Buy IN'
        safsb_buy_in['move_in'] = safsb_buy_in['move_out'] + datetime.timedelta(days=1)
        safsb_buy_in['move_out'] = safsb_buy_in['death_date']
        safsb_buy_in['comment 2'] = 'Buy IN'
        safsb_buy_in['affordable_living'] = 0
        safsb_buy_in['Rental_Units'] = 0
        safsb_buy_in['ApartmentType'] = 'Co-op'
        safsb_buy_in['protected'] = False
        # ridt_mortgage_calc
        monthly_retun = ridt_mortgage_calc.calculte_monthly_mortgage(55000)
        safsb_buy_in['annual_return_mortgage'] = monthly_retun * 12
        safsb_buy_in['annual_tax'] = 0
        safsb_buy_in['annual_tax_burden'] = 0

        safsb_buy_in['annual_hoa_expenses'] = 12 * safsb_buy_in['Ap_sq_feet_size'] * safsb_buy_in['move_in'].apply(ridt_agent_economy.calculate_hoa_for_condos)

        safsb_buy_in['annual_expenses'] = safsb_buy_in['annual_return_mortgage']  + safsb_buy_in['annual_hoa_expenses']
        safsb_buy_in['annual_burden'] = round(
           safsb_buy_in['annual_expenses']/safsb_buy_in['income'], 3)
        ds = pd.concat([ds, safsb_buy_in])
        ds.reset_index(inplace=True, drop=True)
        return ds


## Northtown and Southtown

In [50]:
class ridt_SouthTown:

    @staticmethod
    def generateMarket(cb, Ap_datasets,given_year):
        """generates income range min and max for SouthTown and Northtown
        in addition it generates Upper Income
        cb: current building
        Ap_datasets: property dataset in the current building"""
        if given_year>0:
            year_ = given_year
        else:
            year_ = cb.constr_year
         
        
        affordable_units = cb.affordable_units
        
        if (affordable_units>0):
            # SouthTown Affordable
            Ap_datasets = ridt_SouthTown.south_town_calc_rental(Ap_datasets,cb,year_)
        else:
            """for a given building the function claculates the income"""
            non_affordable = ridt_info.non_affordable_approximation(year_)  
            Ap_datasets['Low_income'] = non_affordable['min']
            Ap_datasets['Upper_income'] = non_affordable['max']
            Ap_datasets['ApartmentType'] = cb.category
            ppf = ridt_info.rent_per_feet_market(year_)
            Ap_datasets['annual_rent'] = Ap_datasets['Ap_sq_feet_size']*ppf*12 #market price
            if (cb.category=='Rental'):
                Ap_datasets['rentalType'] = 'm'
            else:
                Ap_datasets['annual_return_mortgage'] = ridt_agent_economy.update_mortgage_for_year(Ap_datasets,0)
                Ap_datasets['annual_common_charges'] = Ap_datasets['Ap_sq_feet_size'] *ridt_agent_economy.commonChargesPerFeet*12 #Common charges approximantly 1.2 * house size
        return Ap_datasets

    @staticmethod
    def south_town_calc_rental(Ap_datasets,cb,year_):
        """Calc SouthTown Apartment Income according to ABC"""
        a_ap_percent = ridt_info.affordable_models_southtown['Southtown']['a']['ap_percent']
        b_ap_percent = ridt_info.affordable_models_southtown['Southtown']['b']['ap_percent']
        #Batch A (0 - a_max partments)
        a_max = int(np.floor(a_ap_percent*cb.total_units))
        Ap_datasets.loc[0:a_max-1,'rentalType'] ='a'
        #Batch B (a_max-amax+b_max partments)
        b_max = a_max+int(np.floor(b_ap_percent*cb.total_units))
        Ap_datasets.loc[a_max:a_max+b_max-1,'rentalType'] ='b'
        #upper income
        Ap_datasets.loc[a_max+b_max:,'rentalType'] = 'm'
        Ap_datasets = ridt_SouthTown.update_south_town_calc_rental(Ap_datasets,year_)
        return Ap_datasets

    @staticmethod
    def update_south_town_calc_rental(Ap_datasets,year_):
        """Calc SouthTown Apartment Income according to ABC"""
        con_a = (Ap_datasets['rentalType'] =='a')
        #The Logic here is that the rent will be 0.25 of the range of the building
        if len(Ap_datasets[con_a])>0:
            income_min,income_max = ridt_SouthTown.helper_function_2_cal_income('a',year_)
            Ap_datasets.loc[con_a,'Low_income'] = income_min
            Ap_datasets.loc[con_a,'Upper_income'] = income_max
            Ap_datasets.loc[con_a,'annual_rent'] = Ap_datasets.loc[con_a].apply(lambda x: random.randint(x['Low_income'],x['Upper_income'])*0.25,axis=1)
        con_b = (Ap_datasets['rentalType'] =='b')
        if len(Ap_datasets[con_b])>0:
            income_min,income_max = ridt_SouthTown.helper_function_2_cal_income('b',year_)
            Ap_datasets.loc[con_b,'Low_income'] = income_min
            Ap_datasets.loc[con_b,'Upper_income'] = income_max
            Ap_datasets.loc[con_b,'annual_rent'] = Ap_datasets.loc[con_b].apply(lambda x: random.randint(x['Low_income'],x['Upper_income'])*0.25,axis=1)
    
        con_m = Ap_datasets['rentalType']=='m'
        if (len(Ap_datasets[con_m])>0):
            non_affordable = Ap_datasets[con_m].apply(lambda x:ridt_info.get_Income_based_on_marketRent_and_size_year(x['Ap_sq_feet_size'],year_),axis=1)
        
            Ap_datasets.loc[con_m,'Low_income'] = non_affordable['min']
            Ap_datasets.loc[con_m,'Upper_income'] = non_affordable['max']
            Ap_datasets.loc[con_m,'annual_rent'] = non_affordable['min']*0.3
        return Ap_datasets

    @staticmethod
    def helper_function_2_cal_income(a_type,year_):
        """Returns minimum and maximum income"""
        ami_ = ridt_info.simulate_ami_using_polynom(year_)
        ami_min = ridt_info.affordable_models_southtown['Southtown'][a_type]['ami_percent_min']
        ami_max = ridt_info.affordable_models_southtown['Southtown'][a_type]['ami_percent_max']
        income_min = int(ami_min*ami_)
        income_max = int(ami_max*ami_)
        return ([income_min,income_max])

    @staticmethod
    def update_tax_property(ds,year_):
        pass

## Agent

In [51]:
class ridt_agent:
    def helper_function_2_cal_income(a_type,cat,year_):
        """Returns minimum and maximum income"""
        ami_ = ridt_info.simulate_ami_using_polynom(year_)
        ami_min = ridt_info.affordable_models_not_southtown[cat][a_type]['ami_percent_min']
        ami_max = ridt_info.affordable_models_not_southtown[cat][a_type]['ami_percent_max']
        income_min = int(ami_min*ami_)
        income_max = int(ami_max*ami_)
        return ([income_min,income_max])

## Demographic

In [52]:
class ridt_demographic:
    pop_age_dist={'25-35':1,
              '36-45':0.0,
              '46-55':0.0,
              '56-60':0.0,
              '61-65':0.0,
              '66-75':0.0,
              '76-85':0.0,
              '90+':0.00}
    pop_age_mid={'25-35':30,
            '36-45':40,
            '46-55':50,
            '56-60':58,
            '61-65':63,
            '66-75':70,
            '76-85':80,
            '90+':90}
    pop_model_1975_hud_dist = {'25-35':0.55,
            '36-45':0.05,
            '46-55':0.15,
            '56-60':0.15,
            '61-65':0.05,
            '66-75':0.05,
            '76-85':0.00,
            '90+':0.00}
    pop_age_elderlies_dist={'66-75':0.6,'76-85':0.3,'90+':0.1}
    pop_age_elderlies_mid= {'66-75':70,'76-85':80,'90+':90}
    age_group_var = {22:2,30:5,40:5,50:5,58:2,63:2,70:5,80:5,90:5}

    @staticmethod
    def sample_ages(pop_size,choice='other', ag = list(pop_age_mid.values()),p_ = list(pop_age_dist.values())):
        """The Function Returns a sample of Ages according to the given distribtion (With slight variation)
        pop_size: is the size of the poulation (for example pop_size=1000)
        ag: is the age group (for example elderlis 75,85,90)
        p_: is the actual distribution of the group
        """
        if  choice=='hud_1975':
            ag = list(ridt_demographic.pop_age_mid.values())
            p_ = list(ridt_demographic.pop_model_1975_hud_dist.values())
        elif  choice=='Young':
            ag = list(ridt_demographic.pop_age_mid.values())
            p_ = list(ridt_demographic.pop_age_dist.values())
        elif choice=='Old':
            ag = list(ridt_demographic.pop_age_elderlies_mid.values())
            p_ = list(ridt_demographic.pop_age_elderlies_dist.values()) 
        sam = np.random.choice(ag, pop_size, p=p_)
        #Add VAriance
        sam_variance = np.array(list(map(ridt_demographic.age_group_fun, sam)))*np.random.normal(scale=1,size=pop_size)
        sam2 = sam + sam_variance
        return sam2.astype(int)

    def death_date_calculator2(ag,a = 0.0009,b = 0.3313,c=62):
        """for a given AGENT and a A B C parameters
            The Function calculates the Manhattan death Age based on the birthdate
            a one deviation random is calcualted and multipled by 10 (for ten years)
            and that is the offset of the Age
            if the Death Age is less than the entrance age
            than a new random age is given that is the entrance age with 10-15 years offset
        """
        #explanation to Journal + used 100 years statistics + trial and error calibration
        #checked the average age of people who died at certain eyars
        bd = ag['birth_date']
        if 'start_action' in ag.index:
            ed = ag['start_action']
        else:
            ed = ag['move_in']
        y_n = bd.year-1900 #Year normal
        mdy = int(np.power(y_n,2) * a + b*y_n+c) # mdy manhattan death year
        death_date_ = pd.to_datetime(str(bd.year+mdy)+"-1-1")
        random_number_of_years = int(np.random.normal(scale=1)*10) #+- 10 years
        death_date_ = death_date_ +  pd.DateOffset(years=random_number_of_years)
        if (death_date_)<ed:
            death_date_ =  ed +  pd.DateOffset(years=(random.randint(10,15)))
        return death_date_

    @staticmethod
    def age_group_fun(num):
        """get the range of age group given the middle age"""
        return ridt_demographic.age_group_var[num]
        
    @staticmethod
    def generate_new_marketRate_agents_from_avilable_ap(p_ds,bld_name,ai,start_year,comment):
      new_agents = ridt_agents_from_bld.get_agents_from_ap_list(p_ds,bld_name,ai,'Partial_Market',start_year,comment)
      new_agents['annual_return_mortgage'] = new_agents['Ap_sq_feet_size'].apply(lambda x: ridt_mortgage_calc.calculte_monthly_mortgage(x * ridt_agent_economy.pp_sqft)) * 12
      new_agents =ridt_agent_economy.modify_annual_expenses_for_co_op(new_agents,start_year)
      return(new_agents)


## Agent economy

In [53]:
class ridt_agent_economy:
    
    hoa_a = 0.0074
    hoa_b = 1.247 #star year 2000
    utilities_burden = 0.08
    mortgage_burden = 0.43
    pp_sqft = 1000 #dollars per square feet place holder till we will add complexity
    condo_a = 35.714 # ax+b based on cityReality
    condo_b = -71002.857 # ax + b based on city reality
    #https://drive.google.com/file/d/1PoPPN5BI07BvNuAj1EKyzKVAoXGidVKG/view

    co_op_a = 1.833 # ax+b based on cityReality
    co_op_b = -2962.333 # ax + b based on city reality
    #https://drive.google.com/file/d/1pSt6DDOQ1Bahq8KeatDSxx853SZbbQpQ/view
    condo_tax_abatement = 0.25 # based on trial and error needs to be confirmed and calibrated.
    average_tax_rate = 0.125 # average tax rate for condos
    commonChargesPerFeet = 1.2 #Dollar Per sq Feet
    y = str(2000)
    l = ['Q1 '+y,'Q2 '+y,'Q3 '+y,'Q4 '+y]
    d = [y+'-1-1',y+'-4-1',y+'-7-1',y+'-10-1']
    for year_ in range(2001,2100):
        y = str(year_)
        l = l + ['Q1 '+y,'Q2 '+y,'Q3 '+y,'Q4 '+y]
        d = d + [y+'-1-1',y+'-4-1',y+'-7-1',y+'-10-1']
    d = pd.to_datetime(pd.Series(d))
    def month_converter(date_):
        if (date_.month== 12):
            return 1
        elif (date_.month>= 9):
            return 10
        elif (date_.month>= 6):
            return 7
        elif (date_.month>=3):
            return 4
        else:
            return 1
        
    def find_nearest_quarter(date_):
        m = ridt_agent_economy.month_converter(date_) #get nearest month
        if (date_.month)==12:
         y = date_.year+1
        else:
            y = date_.year
        n_date = pd.to_datetime(str(y)+'-'+str(m)+'-1')
        return(n_date)

    @staticmethod
    def get_condo_price_per_feet_based_on_year(y_):
        """get the price per feet based on RI annual increase see:
        https://drive.google.com/file/d/1pSt6DDOQ1Bahq8KeatDSxx853SZbbQpQ/view
        150 dollars randomness was added
        """
        a = ridt_agent_economy.condo_a
        b = ridt_agent_economy.condo_b
        return np.int(y_*a+b + random.randint(-150,150))
        
    def get_co_op_price_per_feet_based_on_year(y_):
        """get the price per feet based on RI annual increase see:
        https://drive.google.com/file/d/1PoPPN5BI07BvNuAj1EKyzKVAoXGidVKG/view
        https://drive.google.com/file/d/1e-JM4_pWFKN8ZWy7xoxcDbb2jmD-ITOt/view

        150 dollars randomness was added
        """
        a = ridt_agent_economy.co_op_a
        b = ridt_agent_economy.co_op_b
        return np.int(y_*a+b + random.randint(-150,150))

    @staticmethod
    def update_mortgage_for_year(ap_ds, y_,coop=False):
        """Update Mortgage based on year and building dataset
        if y_=0 it will take the built year"""
        ap_ds_ = ap_ds.copy()
        if (y_ == 0):
            ppf2 = ap_ds_['built_year'].dt.year.apply(
                ridt_agent_economy.get_condo_price_per_feet_based_on_year)
        else:
            ppf2 = ridt_agent_economy.get_condo_price_per_feet_based_on_year(y_)
        if (coop==True):
            # coop is more expensive
            ppf2 = ppf2 + random.randint(50,200)
        ap_price = ppf2 * ap_ds_['Ap_sq_feet_size']
        ap_ds_['annual_return_mortgage'] = ridt_mortgage_calc.calculte_monthly_mortgage(
            ap_price) * 12
        return ap_ds_['annual_return_mortgage']
    
    @staticmethod
    def update_agents_expenses_staticmethod(ds, cleanupType='Other'):
      """Clean up Agents DS before report"""    
      if (cleanupType=='Rental'):
          con_r = (ds['ApartmentType']=='Rental')
          ds.loc[con_r,'annual_rent_burden'] = ds.loc[con_r,'annual_rent']/ds.loc[con_r,'income']
          ds.loc[con_r,'annual_utilities_burden'] = ds.loc[con_r,'annual_utilities']/ds.loc[con_r,'income']
          ds.loc[con_r,'annual_expenses'] = ds.loc[con_r,'annual_utilities'] + ds.loc[con_r,'annual_rent']
      elif (cleanupType=='Condo'):
          con_c = (ds['ApartmentType']=='Condo')
          ds.loc[con_c,'annual_utilities']=3000
          ds.loc[con_c,'annual_rent']=0
          ds.loc[con_c,'annual_tax']=0
          ds.loc[con_c,'annual_utilities_burden'] = ds.loc[con_c,'annual_utilities']/ds.loc[con_c,'income']
          ds.loc[con_c,'annual_expenses'] = ds.loc[con_c,'annual_return_mortgage'] + ds.loc[con_c,'annual_utilities']+ ds.loc[con_c,'annual_common_charges']
          ds.loc[con_c,'annual_mortgage_burden'] = ds.loc[con_c,'annual_return_mortgage']/ds.loc[con_c,'income']
      else:
          """Converted from rental to Co-ops"""
          con_o = ds['Rental_Units']==0
          ds.loc[con_o,'annual_utilities']=0
          ds.loc[con_o,'annual_rent']=0
          ds.loc[con_o,'annual_tax']=0
          ds.loc[con_o,'annual_common_charges'] = 0
          ds.loc[con_o,'annual_expenses'] = ds.loc[con_o,'annual_return_mortgage'] + ds.loc[con_o,'annual_hoa_expenses']
          ds.loc[con_o,'annual_mortgage_burden'] = ds.loc[con_o,'annual_return_mortgage']/ds.loc[con_o,'income']

      ds['annual_burden'] = ds['annual_expenses']/ds['income']  
      ds['annual_expenses_burden'] = (ds['annual_hoa_expenses'] + ds['annual_utilities']+ds['annual_common_charges'])/ds['income']
      return ds

    @staticmethod
    def find_index_by_year_quarter(date_):
        q_date = ridt_agent_economy.find_nearest_quarter(date_)
        return(ridt_agent_economy.d[ridt_agent_economy.d==q_date].index.values)
    
    @staticmethod
    def calculate_hoa_for_condos(date_):
        x_0 = ridt_agent_economy.find_index_by_year_quarter(date_)
        f_x0 = x_0 * ridt_agent_economy.hoa_a + ridt_agent_economy.hoa_b
        return f_x0[0]
  
    @staticmethod
    def modify_annual_expenses_for_co_op(agents,date_):
      get_cofactor = ridt_agent_economy.calculate_hoa_for_condos(date_)
      agents['annual_hoa_expenses'] = get_cofactor * agents['Ap_sq_feet_size']
      return agents

# Mortgage

In [54]:
class ridt_mortgage_calc:
    i = 0.0438 #annual intrest rate
    i_m = i/12.0 #Monthly Rate
    m_burden = 0.43
    dp = 0.25 #down payment
    ny = 30 #number of years
    n_m = ny*12 # Number of months
    
    def calculte_monthly_mortgage(mv):
        P = mv*(1-ridt_mortgage_calc.dp) #Mortage to ask from bank (25% down payment)
        M = round((P*ridt_mortgage_calc.i_m*np.power((1+ridt_mortgage_calc.i_m),ridt_mortgage_calc.n_m))/(np.power((1+ridt_mortgage_calc.i_m),ridt_mortgage_calc.n_m)-1),0)
        return M

## RIDT_Agents_OBJ

In [55]:
class ridt_agents_obj:
    
    def __init__(self,ds):
        self.ds = ds

    def update_cleanup_agents(self, cleanupType='Other'):
      """Clean up Agents DS before report"""
      self.ds = ridt_agent_economy.update_agents_expenses_staticmethod(self.ds,cleanupType)
      
        
    def filter_ds_by_year(self,year_,onlyLiving=False):
        ds = self.ds.copy()
        if (onlyLiving==True):
          date_ = pd.to_datetime(str(year_)+"-1-1")
          
          sample_ds = ridt_agents_obj(
              ds[((ds['move_in'] <= (date_)) & 
                       (ds['death_date']>
                                          (date_))) &
                      ((ds['move_in'] <= (date_)) & (ds['move_out']>(date_)))
                                        ])
          return sample_ds
        else:
          date_ = pd.to_datetime(str(year_)+"-1-1")
          sample_ds = ridt_agents_obj(
              ds[(ds['move_in'] <= (date_)) & 
                                        (ds['move_out']>
                                          (date_))])
          return sample_ds


    
    def filter_ds_by_full_year_date(self,start_year):
      end_of_year = pd.to_datetime(str(start_year.year+1)+"-1-1")
      sample_ds = ridt_agents_obj(self.ds[(self.ds['move_in'] <= (start_year)) & (self.ds['move_out']>=(end_of_year))])
      return sample_ds

    def drop(self,dr_ds,field_):
      """Sepcial Drop, drop by feild for this object"""
      self.ds = (self.ds[~self.ds[field_].isin(dr_ds[field_])]).reset_index(drop=True)
    
    def concat(self, ds_array):
      """Special Concat for a list of arrawys"""
      for ds in ds_array:
        self.ds = pd.concat([self.ds,ds])
      self.ds.reset_index(drop=True,inplace=True)

    def drop_future_agent_cycles(self,safsb_buy_out):
        """receives the buy-out tenants and deltes all future of their cycle
        safsb_buy_out: is the tenants who are buying out"""
        #self.ds['address_door']==
        for door,cycle in zip(safsb_buy_out['address_door'],safsb_buy_out['tenant_cycle']):
            #print('door '+door)
            #print('cycle '+ str(cycle))
            d = self.ds.loc[(self.ds['address_door']==door) &(self.ds['tenant_cycle']>cycle)]
            #print(list(d.index))
            self.ds.drop(list(d.index),inplace=True)
            #self.ds = self.ds[~self.ds.index.isin(list(d.index))]

    def set_income_based_on_range(self):
      """Applies income based on the current Low and High of the building Category Conditions"""
      self.ds['income'] = self.ds.apply(lambda x: random.randint(int(x['Low_income']),int(x['Upper_income'])),axis=1)


## Agent from building

In [56]:
class ridt_agents_from_bld(ridt_agents_obj):

    def __init__(self,bldgs_ds, option,year_):
        """Creates the original object of agents"""
        #I can use the Function get_standard_agents_from_ap_list to cut on code
        self.ds = bldgs_ds.copy()
        self.basic_init()
        if (option=='all_init'):
          self.init_all()
        elif (option=='copy'):
          self.simple_init(year_)
          #addfunction to modify income according to year

    def basic_init(self):
        """default init for buildings"""
        self.ds['agent_action'] = 'In Building'
        self.ds['cause'] = 'death'
        self.ds['annual_hoa_expenses'] = 0
        self.ds['annual_tax'] = 0
        self.ds['annual_rent'] = 0            
        self.ds['annual_expenses'] = 0
        self.ds['annual_burden'] = 0
        self.ds['annual_tax_burden'] = 0
        self.ds['annual_rent_burden'] = 0
        self.ds['annual_utilities_burden'] = 0
        self.ds['annual_common_burden'] = 0
        self.ds['annual_mortgage_burden'] = 0
        self.ds['annual_expenses_burden'] = 0
        self.ds['agentID'] = [uuid.uuid4() for _ in range(len(self.ds.index))] #new agent ID number
        self.ds['previous_agentID']='0'
        self.ds['tenant_cycle'] =0
        self.ds['comment 1']=''
        self.ds['comment 2']=''
        self.ds['current_year']=''
        self.ds['current_age']=''   
        self.ds['income']  = 0
        
    def simple_init(self,year_):
      """"""
      move_in_d = pd.to_datetime(str(year_)+'-5-2')
      self.ds['move_in'] = move_in_d

      ### Setting Birth Day for old Ages
      con = self.ds['elderly_lving']==1 #old age condition
      count_elderlies = len(self.ds[con].index)
      if count_elderlies>0:
        old_ages = ridt_demographic.sample_ages(count_elderlies,'Old')
        self.ds.loc[con,'birth_date'] = self.ds.loc[con,'move_in'].dt.year - old_ages

      ###Setting Birthday for Youngsters
      con = self.ds['elderly_lving']==0 #old age condition
      count_young = len(self.ds[con].index)
      young_ages = ridt_demographic.sample_ages(count_young,'Young')
      self.ds.loc[con,'birth_date'] = self.ds.loc[con,'move_in'].dt.year - young_ages
      self.ds['birth_date'] = self.ds['birth_date'].astype(int)
      self.ds['birth_date'] = self.ds['birth_date'].apply(lambda x: str(x)+"-5-1")
      self.ds['birth_date'] = pd.to_datetime(self.ds['birth_date'])
      #death age
      self.ds['death_date'] = self.ds.apply(ridt_demographic.death_date_calculator2,axis=1)
      self.ds['death_age'] = self.ds['death_date'].dt.year-self.ds['birth_date'].dt.year
      self.ds['entrance_age'] = self.ds['move_in'].dt.year-self.ds['birth_date'].dt.year
      self.ds['move_out'] = self.ds['death_date']
      
    def init_all(self):
      """"""
      self.ds['move_in'] = self.ds['built_year']
      ### Setting Birth Day for old Ages
      con = self.ds['elderly_lving']==1 #old age condition
      count_elderlies = len(self.ds[con].index)
      old_ages = ridt_demographic.sample_ages(count_elderlies,'Old')
      self.ds.loc[con,'birth_date'] = self.ds.loc[con,'move_in'].dt.year - old_ages

      ###Setting Birthday for Youngsters
      con = self.ds['elderly_lving']==0 #old age condition
      count_young = len(self.ds[con].index)
      young_ages = ridt_demographic.sample_ages(count_young,'hud_1975')
      self.ds.loc[con,'birth_date'] = self.ds.loc[con,'move_in'].dt.year - young_ages
      
      self.ds['birth_date'] = self.ds['birth_date'].apply(int)
      self.ds['birth_date'] = self.ds['birth_date'].apply(lambda x: str(x)+"-"+str(random.randint(1,12))+"-"+str(random.randint(1,28)))
      self.ds['birth_date'] = pd.to_datetime(self.ds['birth_date'])
      #death age
      self.ds['death_date'] = self.ds.apply(ridt_demographic.death_date_calculator2,axis=1)
      self.ds['death_age'] = self.ds['death_date'].dt.year-self.ds['birth_date'].dt.year
      self.ds['entrance_age'] = self.ds['move_in'].dt.year-self.ds['birth_date'].dt.year
      self.ds['move_out'] = self.ds['death_date']
      self.ds['Low_income'] = self.ds['Low_income'].astype(int)
      self.ds['Upper_income'] = self.ds['Upper_income'].astype(int)
      self.ds['income']  = self.ds.apply(lambda x: random.randint(x['Low_income'],x['Upper_income']),axis=1)

    @staticmethod
    def get_agents_from_ap_list(bld_obj,bld_name,ap_list,category,move_in_date,comment):
      """The Function Recieves:
      bldg_obj(pds) to generate standerd agent for door number
      Bld_name (needs to replace to bld_num)
      ap_list the index of the door
      Building Category
      move_in_date
      Comment
      """
      if (category=='Partial_Market'):
        subset = bld_obj.ds[bld_obj.ds['Building Name'] == bld_name].copy()
        sb = subset[subset['ap_index'].isin(ap_list)]
      else:
        pass
        #sb = bld_obj.get_ap_data_from_specific_building_num(bld_name,ap_list)
        
      ads = ridt_agents_from_bld.get_standard_agents_from_ap_list(sb,move_in_date)
      if (category=='Partial_Market'):
        print('in Co-op')
        ads['affordable_living'] = 0
        ads['comment 1'] = comment
        ads['Rental_Units'] = 0
        ads['protected'] = False
        ads['ApartmentType'] = 'Co-op'
      elif (category=='Condos'):
        ads['affordable_living'] = 0
        ads['comment 1'] = comment
        ads['Rental_Units'] = 0
        ads['protected'] = False
        ads['ApartmentType'] = 'Condo'
      else:
        ads['comment 1'] = comment
        ads['Rental_Units'] = 1
        ads['protected'] = False
        ads['ApartmentType'] = 'Rental'
      return ads

    @staticmethod
    def get_standard_agents_from_ap_list(sb,move_in_date,bld_type='Partial_Market'):
      #I can merge both functions! (Init and this funciton)
      ads = sb.copy()
      ads['agent_action'] = 'In Building'
      ads['move_in'] = move_in_date
      if (bld_type=='Partial_Market'):
        count_pop = len(ads.index)
        young_ages = ridt_demographic.sample_ages(count_pop,'Young')
        ads['birth_date'] = ads['move_in'].dt.year - young_ages
      else:
        ads = ridt_agents_from_bld.birth_date_diff(ads)
      ads['birth_date'] = ridt_agents_from_bld.fix_birthdate(ads)
      #death age
      ads['death_date'] = ads.apply(ridt_demographic.death_date_calculator2,axis=1)
      ads['death_age'] = ads['death_date'].dt.year-ads['birth_date'].dt.year
      ads['entrance_age'] = ads['move_in'].dt.year-ads['birth_date'].dt.year
      ads['move_out'] = ads['death_date']
      ads['cause'] = 'death'
      years_ = ads['move_in'].dt.year
      ami_ = years_.apply(ridt_info.simulate_ami_using_polynom)
      ami_low = ami_*(2/3)
      ami_high = ami_*(4/3)
      ads.loc[ads['Low_income'].isnull(),'Low_income'] = ami_low.loc[ads['Low_income'].isnull()]
      ads.loc[ads['Upper_income'].isnull(),'Upper_income'] = ami_high.loc[ads['Upper_income'].isnull()]
      ads['Low_income'] = ads['Low_income'].astype(int)
      ads['Upper_income'] = ads['Upper_income'].astype(int)
      ads['income']  = ads.apply(lambda x: random.randint(x['Low_income'],x['Upper_income']),axis=1)
      ads['annual_utilities'] = 0
      ads['annual_return_mortgage'] = 0
      ads['annual_hoa_expenses'] = 0
      ads['annual_utilities_burden'] = 0           
      ads['annual_expenses'] = 0
      ads['annual_burden'] = 0
      ads['annual_tax'] = 0
      ads['annual_tax_burden'] = 0
      ads['agentID'] = [uuid.uuid4() for _ in range(len(ads.index))]
      ads['previous_agentID']='0'
      ads['tenant_cycle'] =0
      ads['comment 1']=''
      ads['comment 2']=''      
      return ads

    @staticmethod
    def fix_birthdate(ads,field_='birth_date'):
      ads1 = ads.copy()
      ads1[field_] = ads1[field_].apply(int)
      ads1[field_] = ads1[field_].apply(lambda x: str(x)+"-"+str(random.randint(1,12))+"-"+str(random.randint(1,28)))
      return (pd.to_datetime(ads1[field_]))

    @staticmethod
    def birth_date_diff(ads):
        ### Setting Birth Day for old Ages
        con = ads['elderly_lving']==1 #old age condition
        count_elderlies = len(ads[con].index)
        old_ages = ridt_demographic.sample_ages(count_elderlies,'Old')
        ads.loc[con,'birth_date'] = ads.loc[con,'move_in'].dt.year - old_ages
        #### Youngsters
        con = ads['elderly_lving']==0 #old age condition
        count_young = len(ads[con].index)
        young_ages = ridt_demographic.sample_ages(count_young,'Young')
        ads.loc[con,'birth_date'] = ads.loc[con,'move_in'].dt.year - young_ages
        return ads


      

## ridt_agents_ts_obj ⏳

In [57]:
class ridt_agents_ts_obj(ridt_agents_obj):
    last_date = pd.to_datetime((datetime.date(2100, 1, 1)))
    def __init__(self,agents_ds):
        self.ds = agents_ds.copy()

    def detach_by_list_of_bld_keys(self,list_of_Bldg):
        work_ds = ridt_agents_ts_obj(self.ds.loc[self.ds['bld_key'].isin(list_of_Bldg)].copy())
        main_ds =   ridt_agents_ts_obj(self.ds.loc[~self.ds['bld_key'].isin(list_of_Bldg)].copy())
        return([main_ds,work_ds])
    def filter_ds_by_year_detach(self,year_):
      this_year = self.filter_ds_by_year(year_).ds.copy()
      future_years = self.ds.loc[~self.ds.index.isin(this_year.index)].copy()
      this_year = ridt_agents_ts_obj(this_year.reset_index(drop=True))
      future_years = ridt_agents_ts_obj(future_years.reset_index(drop=True))
      return ([this_year,future_years])

    def get_burden_agents(self,BurdenType):
      """get rent burden"""
      burden_agents = self.ds[self.ds['comment 1']==BurdenType]
      return burden_agents

    def remove_dead_agents_for_year(self,start_year):
      """return dead agents of this year"""
      y_ = start_year.year
      start_year = pd.to_datetime(str(y_)+"-1-1")
      end_of_year = pd.to_datetime(str(y_)+"-12-31")
      #end_of_year = pd.to_datetime(str(start_year.year+1)+"-1-1")
      dead_agents = self.ds[(self.ds['death_date']>=start_year) &(self.ds['death_date']<end_of_year)]
      return dead_agents
    
    def find_agents_to_remove_market_rate_for_year(self,start_year):
      """"""
      end_of_year = pd.to_datetime(str(start_year.year+1)+"-1-1")
      middle_of_year = pd.to_datetime(str(start_year.year)+"-5-1")
      ag_sub = self.filter_ds_by_full_year_date(start_year) # year subset
      dead_agents = self.ds[(self.ds['death_date']>=start_year) &(self.ds['death_date']<end_of_year)].copy() # Dead
      dead_agents['cause'] = 'death'
      ag_sub = ag_sub.ds[~ag_sub.ds['agentID'].isin(dead_agents['agentID'])] # remove dead agents
      #Need to update the conditions to match 
      ub = ridt_agent_economy.utilities_burden
      mb = ridt_agent_economy.mortgage_burden
      #utilities_burden_ag = (ag_sub.ds[ag_sub.ds['annual_expenses_burden']>=ub]).copy()
      mortgage_burden_age = (ag_sub[ag_sub['annual_mortgage_burden']>=mb]).copy()
      mortgage_burden_age['cause'] = 'Mortgage Burden'
      ag_sub = ag_sub[~ag_sub['agentID'].isin(mortgage_burden_age['agentID'])] # remove mortage agents
      expenses_bruden = (ag_sub[ag_sub['annual_expenses_burden']>=mb]).copy()
      expenses_bruden['cause'] ='Total Burden'
      ag_sub = ag_sub[~ag_sub['agentID'].isin(expenses_bruden['agentID'])] # remove mortage agents
      # Need to replace utilites with total exepenses
      #burden_agents = pd.concat([utilities_burden_ag,mortgage_burden_age])
      Agents_to_displace = pd.concat([dead_agents,mortgage_burden_age,expenses_bruden])
      Agents_to_displace['move_out'] = middle_of_year
      #Agents_to_displace.drop_duplicates(inplace=True)
      return Agents_to_displace

    def filter_ds_by_building(self, bld):
      return ridt_agents_obj(self.ds[self.ds['Building Name']==bld])
    
    def copy(self):
      """Copy Myself"""
      return ridt_agents_ts_obj(self.ds)
      
    @staticmethod
    def replace_dead_tenants_once(datr):
        #step1 save move_out --> move_in
        datr['move_in'] =datr['move_out'] + datetime.timedelta(days=1)
        ### Setting Birth Day for old Ages
        con = datr['elderly_lving']==1 #old age condition
        count_elderlies = len(datr[con].index)
        old_ages = ridt_demographic.sample_ages(count_elderlies,'Old')
        datr.loc[con,'birth_date'] = datr.loc[con,'move_in'].dt.year - old_ages
        con = datr['elderly_lving']==0 #Young age condition
        count_young = len(datr[con].index)
        young_ages = ridt_demographic.sample_ages(count_young,'Young')
        datr.loc[con,'birth_date'] = datr.loc[con,'move_in'].dt.year - young_ages

        datr['birth_date'] = datr['birth_date'].apply(int)
        datr['birth_date'] = datr['birth_date'].apply(lambda x: str(x)+"-"+str(random.randint(1,12))+"-"+str(random.randint(1,28)))
        datr['birth_date'] = pd.to_datetime(datr['birth_date'])
        datr['death_date'] = datr.apply(ridt_demographic.death_date_calculator2,axis=1)
        con = datr['death_date']> ridt_agents_ts_obj.last_date
        datr.loc[con,'death_date'] = ridt_agents_ts_obj.last_date
        datr['death_age'] = datr['death_date'].dt.year-datr['birth_date'].dt.year
        datr['entrance_age'] = datr['move_in'].dt.year-datr['birth_date'].dt.year
        datr['move_out'] = datr['death_date']
        minmax =  datr['move_in'].dt.year.apply(ridt_info.non_affordable_approximation)
        min_ =[]
        max_ =[]
        for i in minmax.index:
            min_.append(int(minmax[i]['min']))
            max_.append(int(minmax[i]['max']))
        datr['Low_income'] = min_
        datr['Upper_income'] = max_
        datr['income']  = datr.apply(lambda x: random.randint(x['Low_income'],x['Upper_income']),axis=1)
        #App_datasets['income'] =  App_datasets.apply(lambda x: int(random.uniform(1.48,2.0)*ridt_info.ami_100_percent), axis=1) 
        datr['previous_agentID']=datr['agentID']
        datr['agentID'] = [uuid.uuid4() for _ in range(len(datr.index))]
        datr['tenant_cycle'] +=1
        return datr

    @staticmethod
    def update_cause_of_departure(dapartedAgents,departure_date,cause):
      """update agents to be displace date of departure and cause (for one category only)"""
      dapartedAgents['move_out'] = departure_date
      dapartedAgents['cause'] = cause
      if (cause=='death'):
          dapartedAgents['death_date'] = departure_date
      return dapartedAgents

    @staticmethod
    def update_status_of_displaced_agents_market_rate(Agents_to_displace,start_year):
      """"""
      Agents_to_displace['move_out'] = start_year
      Agents_to_displace['cause'] = Agents_to_displace.apply(ridt_demographic.cause_of_displacement, axis=1)
      return Agents_to_displace



## Widgets

In [102]:
class ridt_widgets:
    bldg_json_wid = widgets.Text(value='https://raw.githubusercontent.com/Shai2u/RI_Microsimulator_July2021/main/Simulation/json/bldgs_nov_16_2050.geojson' ,description='GeoJson Buildings File :', style = {'description_width': 'initial'},layout=Layout(width='75%'))
    wire_data_excel = widgets.Text(value='https://raw.githubusercontent.com/Shai2u/RI_Microsimulator_July2021/main/Simulation/wire_apt_rooms_dist_28_9.xlsx',description='Wire Data Excel: ', style = {'description_width': 'initial'},layout=Layout(width='75%'))
    bldg_event_excel = widgets.Text(value='https://raw.githubusercontent.com/Shai2u/RI_Microsimulator_July2021/main/Simulation/building_event_dec_21.xlsx',description='Building Event Excel: ', style = {'description_width': 'initial'},layout=Layout(width='75%'))
    def widget1():
        """Function to display the first set of widgets"""
        display(ridt_widgets.bldg_json_wid,
        ridt_widgets.wire_data_excel,
        ridt_widgets.bldg_event_excel)
        json_path = ridt_widgets.bldg_json_wid.value
        json_bldgs = gpd.read_file(json_path, driver='GeoJSON')
        wire_bldg_data = pd.read_excel(ridt_widgets.wire_data_excel.value)
        bldg_events = pd.read_excel(ridt_widgets.bldg_event_excel.value)
        return([json_bldgs,wire_bldg_data,bldg_events])

# Building Island Timeline
## Load Buildings and Events 

In [103]:
json_bldgs,wire_bldg_data,bldg_events = ridt_widgets.widget1()

Text(value='https://raw.githubusercontent.com/Shai2u/RI_Microsimulator_July2021/main/Simulation/json/bldgs_nov…

Text(value='https://raw.githubusercontent.com/Shai2u/RI_Microsimulator_July2021/main/Simulation/wire_apt_rooms…

Text(value='https://raw.githubusercontent.com/Shai2u/RI_Microsimulator_July2021/main/Simulation/building_event…

# Building Apartment Listing
# 🏠🏢🏠🏢🏠🏢🏠🏢🏠🏢🏠🏢🏠🏢🏠🏢

In [106]:
p_ds = ridt_properties(json_bldgs) # build an object for the apartments



bld name island house
wbd_subset
bld name Roosevelt Landings
wbd_subset
bld name Rivercross
wbd_subset


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


bld name Westview
wbd_subset


In [107]:
# Example of apartment listing
from ipywidgets import HTML
display(HTML(value="<h1>Display part of the listing 🏠</h1>"))
p_ds.ds.head()

HTML(value='<h1>Display part of the listing 🏠</h1>')

,ap_index,bld_key,address_door,Building Name,built_year,affordable_living,elderly_lving,Group,Rental_Units,rentalType,ApartmentType,annual_utilities,Ap_sq_feet_size,Ap_number_of_rooms,protected,Low_income,Upper_income,annual_rent,annual_return_mortgage,annual_common_charges
0,0,10-40,10-40_0,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560.371875,107200.875,17688.0,NaN,NaN
1,1,10-40,10-40_1,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560.371875,107200.875,17688.0,NaN,NaN
2,2,10-40,10-40_2,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560.371875,107200.875,17688.0,NaN,NaN
3,3,10-40,10-40_3,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560.371875,107200.875,17688.0,NaN,NaN
4,4,10-40,10-40_4,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560.371875,107200.875,17688.0,NaN,NaN


In [108]:
#Inital Agents
display(HTML(value="<h1>Initalizing Agents (No Time invovled yet) 🏠</h1>"))
agents_ds_1 = ridt_agents_from_bld(p_ds.ds,option='all_init',year_=0)
df_ts_1 = ridt_agents_ts_obj(agents_ds_1.ds)
df_ts_1.ds.head()

HTML(value='<h1>Initalizing Agents (No Time invovled yet) 🏠</h1>')

,ap_index,bld_key,address_door,Building Name,built_year,affordable_living,elderly_lving,Group,Rental_Units,rentalType,ApartmentType,annual_utilities,Ap_sq_feet_size,Ap_number_of_rooms,protected,Low_income,Upper_income,annual_rent,annual_return_mortgage,annual_common_charges,agent_action,cause,annual_hoa_expenses,annual_tax,annual_expenses,annual_burden,annual_tax_burden,annual_rent_burden,annual_utilities_burden,annual_common_burden,annual_mortgage_burden,annual_expenses_burden,agentID,previous_agentID,tenant_cycle,comment 1,comment 2,current_year,current_age,income,move_in,birth_date,death_date,death_age,entrance_age,move_out
0,0,10-40,10-40_0,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560,107200,0,NaN,NaN,In Building,death,0,0,0,0,0,0,0,0,0,0,911a4373-08e2-4581-af23-5752c3a64a5d,0,0,,,,,105388,1989-01-01,1967-01-11,2050-01-01,83,22,2050-01-01
1,1,10-40,10-40_1,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560,107200,0,NaN,NaN,In Building,death,0,0,0,0,0,0,0,0,0,0,2e99cbb7-fe74-40f3-a20a-3f526f65d682,0,0,,,,,73275,1989-01-01,1963-05-02,2053-01-01,90,26,2053-01-01
2,2,10-40,10-40_2,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560,107200,0,NaN,NaN,In Building,death,0,0,0,0,0,0,0,0,0,0,b48cac5c-351c-496d-9995-4796f800bf40,0,0,,,,,47099,1989-01-01,1935-06-13,2016-01-01,81,54,2016-01-01
3,3,10-40,10-40_3,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560,107200,0,NaN,NaN,In Building,death,0,0,0,0,0,0,0,0,0,0,f9e2cddf-d45e-4c49-a79f-c9f39f036ba7,0,0,,,,,70511,1989-01-01,1951-04-04,2024-01-01,73,38,2024-01-01
4,4,10-40,10-40_4,Manhattan park,1989-01-01,0,0,NorthTown,1,m,Rental,3000,1100.0,2.0,False,45560,107200,0,NaN,NaN,In Building,death,0,0,0,0,0,0,0,0,0,0,6c1b203c-3faa-4bec-94e0-8728fb67c2d0,0,0,,,,,98730,1989-01-01,1930-08-21,2005-01-01,75,59,2005-01-01


# Adding Market Simulation for Non WIRE buildings

In [109]:
def iterate_one_year_SouthTownRental(y_,work_ds):
    se_ = pd.to_datetime(str(y_)+'-1-1') # start event
    year_subset = work_ds.filter_ds_by_year(y_) #get agents for the current year
    year_subset.ds = ridt_SouthTown.update_south_town_calc_rental(year_subset.ds,y_) #update rental prices and ranges
    if len(year_subset.ds)==0:
        print('return year return 0')
        return
    #update rental prices in working dataset
    year_subset.update_cleanup_agents(cleanupType='Rental')
    #----------------------------------------------------------
    #check for agents who died
    dead_agents = work_ds.remove_dead_agents_for_year(se_).copy()
    dead_agents = dead_agents[dead_agents['comment 1']=='']
    dead_agents = ridt_agents_ts_obj.update_cause_of_departure(dead_agents,pd.to_datetime(str(y_)+'-5-1'),'death') #update case of displaccement
    dead_agents['comment 1'] = 'death'
    year_subset.drop(dead_agents,'agentID')  
    #drop displaced Agents

    #---------------------------------------------------------
    #check house prices and update burden and mark potential burden agents
    burden_mark_for_year = mark_rent_burden(ridt_agents_obj(calcRentalBurdenForYear(y_,year_subset.ds)))
    burden_mark_for_year = ridt_agents_ts_obj(burden_mark_for_year.ds)
    burden_agents = burden_mark_for_year.get_burden_agents('Rental Burden').copy()
    burden_agents = ridt_agents_ts_obj.update_cause_of_departure(burden_agents,pd.to_datetime(str(y_)+'-5-1'),'Rent Burden')#update case of displaccement
    year_subset = ridt_agents_ts_obj(year_subset.ds)
    year_subset.drop(burden_agents,'agentID') #drop displaced Agents

    # ----------------------------------------------------------
    # check Other Accumulated Burdens
    total_burden_for_year = mark_burden(year_subset.ds, 0.43, 'Total')

    total_burden_for_year = ridt_agents_ts_obj(total_burden_for_year)

    total_burden = total_burden_for_year.get_burden_agents(
        'Total Burden').copy()
    total_burden = ridt_agents_ts_obj.update_cause_of_departure(
        total_burden, pd.to_datetime(str(y_)+'-5-1'), 'Total Burden')  # update case of displaccement
    year_subset.drop(total_burden, 'agentID')  # drop displaced Agents

    #----------------------------------------------------------
    #Drop displaced agents from main sets also in future cycles
    a2remove = pd.concat([dead_agents,burden_agents,total_burden]) #agents to remove
    if len(a2remove)==0:
        print('return a2remove is zero')
        return
    work_ds.drop(year_subset.ds,'agentID')
    work_ds.drop(dead_agents,'agentID')
    work_ds.drop(a2remove,'agentID') #drop displaced Agents
    print('len of agents to drop', len(a2remove))
    #----------------------------------------------------------
    #generate new agents in the pool of rental doors
    bld_pool = list(year_subset.ds['bld_key'].unique())    
    occupied = list(year_subset.ds['address_door'])
    pool_of_apartments = p_ds.ds[p_ds.ds['bld_key'].isin(bld_pool)].copy()
    pool_of_doors = list(pool_of_apartments[pool_of_apartments['ApartmentType']=='Rental']['address_door'])
    avialable_doors = [door for door in pool_of_doors if door not in occupied]
    vacant_apartments = pool_of_apartments[pool_of_apartments['address_door'].isin(avialable_doors)].copy()
    print('len of vacant_apartments', len(vacant_apartments))
    #--------------------------------------------------------------------------------------------------
    new_agents = ridt_agents_from_bld(vacant_apartments,option='copy',year_=y_)
    new_agents_1 = ridt_SouthTown.update_south_town_calc_rental(new_agents.ds,y_)
    new_agents_1_obj = ridt_agents_obj(new_agents_1)
    new_agents_1_obj.set_income_based_on_range()
    new_agents_2 = addPreviousTenantHistory(
    new_agents_1_obj.ds, a2remove)
    #a2remove.reset_index(inplace=True,drop=True)
    #new_agents_2.reset_index(inplace=True,drop=True)
    new_agents_2 = ridt_agent_economy.update_agents_expenses_staticmethod(new_agents_2, cleanupType='Rental')
    print('len of new agents', len(new_agents_2))
    #-------------------------------------------------------------
    work_ds.concat([year_subset.ds,a2remove,new_agents_2])
    #work_ds
    work_ds.ds.reset_index(inplace=True,drop=True)
    return (work_ds)


In [110]:
def calcRentalBurdenForYear(year_,w_subset):
    """This function recieves year, price per fe and returns the rent burden"""
    w_subset_ds = w_subset.copy()
    w_subset_ds['current_year'] =year_
    w_subset_ds['annual_rent_burden'] = (w_subset_ds['annual_rent'] / w_subset_ds['income'])
    return (w_subset_ds)


In [111]:
def calcRentalUtilitiesBurdenForYear(year_,w_subset):
    """This function recieves year, price per fe and returns the rent burden"""
    w_subset_ds = w_subset.copy()
    w_subset_ds['current_year'] =year_
    w_subset_ds['annual_utilities_burden'] = (w_subset_ds['annual_utilities'] / w_subset_ds['income'])
    return (w_subset_ds)

In [112]:
def mark_rent_burden(ds,threshold=0.3):
    ds.ds.loc[ds.ds['annual_rent_burden']>=threshold,'comment 1'] = 'Rental Burden'
    return(ds)

In [113]:
def mark_utlities_burden(ds,threshold=0.1):
    ds.ds.loc[ds.ds['annual_expenses_burden']>=threshold,'comment 1'] = 'Utilities Burden'
    return(ds)

In [114]:
def mark_burden(ds,threshold, cause):
    """If Mortgage Burden is above 43% Mark it"""
    if (cause =='Mortgage'):
        ds.loc[ds['annual_mortgage_burden']>=threshold,'comment 1'] = 'Mortgage Burden'
    elif (cause == 'Total'):
        ds.loc[ds['annual_burden']>=threshold,'comment 1'] = 'Total Burden'
    return(ds)

In [115]:
def update_south_town_calc_condo(ap_ds, year_):
    """Update new Condo Tenants (need to check for duplication to make more efficeint
    ap_ds is the agents dataset
    year_ is the current year"""
    #!!!!! Might need to add an inflation or HOA calibrator to make pricing more expensive year after year
    ap_ds['annual_return_mortgage'] = ridt_agent_economy.update_mortgage_for_year(ap_ds,year_)#update Motrage
    ap_ds['annual_common_charges'] =  ap_ds['Ap_sq_feet_size'] *ridt_agent_economy.commonChargesPerFeet*12 #Common charges approximantly 1.2 * house size  #Tax
    ap_ds['annual_utilities'] = 3000 #Utilities
    ap_ds['annual_expenses'] = (ap_ds['annual_utilities']+ap_ds['annual_common_charges'] + ap_ds['annual_return_mortgage']).astype(int)
    ap_ds['current_year'] =year_
    min_ = (ap_ds['annual_expenses']/0.6).astype(int)
    ### QUESTION FOR SHARON!!!
    max_ = (ap_ds['annual_expenses']/0.1).astype(int)
    ap_ds_copy = ap_ds.copy()
    ap_ds_copy['min_'] = min_
    ap_ds_copy['max_'] = max_
    ap_ds['income'] = ap_ds_copy.apply(lambda x: random.randint(x['min_'],x['max_']),axis=1)
    ap_ds = ridt_agent_economy.update_agents_expenses_staticmethod(ap_ds,"Condo")
    return ap_ds


In [116]:
def addPreviousTenantHistory(new_agents,Agents_to_displace):
  na =new_agents.copy()
  col_to_remove =['address_door','tenant_cycle','agentID','cause']
  ad_slim = Agents_to_displace[col_to_remove].copy()
  ad_slim = ad_slim[ad_slim['agentID']!='']
  ad_slim = ad_slim.drop_duplicates()
  col_to_remove_x = [col + "_x" for col in col_to_remove]
  na_2 = pd.merge(na,ad_slim, how='left',on='address_door',suffixes=('','_x'))
  #na_2['tenant_cycle'] = (na_2['tenant_cycle_x']+1).fillna(0)
  #na_2['previous_agentID'] = (na_2['agentID_x']).fillna('')
  na_2['tenant_cycle'] = (na_2['tenant_cycle_x']+1)
  na_2['previous_agentID'] = (na_2['agentID_x'])
  na_2['tenant_cycle'] = na_2['tenant_cycle'].fillna(0)
  na_2['previous_agentID'] = na_2['previous_agentID'].fillna('')
  na_2.drop(columns=col_to_remove_x[1:],inplace=True)
  return (na_2)

In [117]:
def south_town_rent_updater(df_ts, GroupBuildings,last_year):
    query_str = "(Category=='Rental')&(Group=='"+GroupBuildings+"')"
    rental_south_town = p_ds.bldg_dict.query(query_str)
    #get the list of southtown buildings
    rental_bldg_list = list(rental_south_town['bldgs_id'].values)
    #detach all dataset with southtown rental dataset
    main_ds, work_ds = ridt_agents_ts_obj(df_ts.ds).detach_by_list_of_bld_keys(rental_bldg_list)
    #make a list of years to modify
    #condos_ds, renters_ds
    work_ds = ridt_agents_ts_obj(work_ds.ds)
    work_ds.update_cleanup_agents('Rental')
    years_range = range(work_ds.ds['built_year'].min().year,last_year)
    for y_ in years_range:
        print('year',y_)
        work_ds = iterate_one_year_SouthTownRental(y_,work_ds)
    return(ridt_agents_ts_obj(pd.concat([main_ds.ds,work_ds.ds]).reset_index(drop=True)))

## Declartion

In [118]:
df_ts_3 = south_town_rent_updater(df_ts_1, 'SouthTown',2070)
df_ts_4 = south_town_rent_updater(df_ts_3, 'NorthTown',2070)

year 2004
len of agents to drop 12
len of vacant_apartments 12
len of new agents 12
year 2005
len of agents to drop 21
len of vacant_apartments 21
len of new agents 21
year 2006
len of agents to drop 28
len of vacant_apartments 28
len of new agents 28
year 2007
len of agents to drop 41
len of vacant_apartments 41
len of new agents 41
year 2008
len of agents to drop 35
len of vacant_apartments 35
len of new agents 35
year 2009
len of agents to drop 39
len of vacant_apartments 39
len of new agents 39
year 2010
len of agents to drop 51
len of vacant_apartments 51
len of new agents 51
year 2011
len of agents to drop 51
len of vacant_apartments 51
len of new agents 51
year 2012
len of agents to drop 33
len of vacant_apartments 33
len of new agents 33
year 2013
len of agents to drop 52
len of vacant_apartments 52
len of new agents 52
year 2014
len of agents to drop 53
len of vacant_apartments 53
len of new agents 53
year 2015
len of agents to drop 69
len of vacant_apartments 69
len of new ag

len of new agents 128
year 2019
len of agents to drop 121
len of vacant_apartments 121
len of new agents 121
year 2020
len of agents to drop 114
len of vacant_apartments 114
len of new agents 114
year 2021
len of agents to drop 137
len of vacant_apartments 137
len of new agents 137
year 2022
len of agents to drop 102
len of vacant_apartments 102
len of new agents 102
year 2023
len of agents to drop 95
len of vacant_apartments 95
len of new agents 95
year 2024
len of agents to drop 109
len of vacant_apartments 109
len of new agents 109
year 2025
len of agents to drop 118
len of vacant_apartments 118
len of new agents 118
year 2026
len of agents to drop 133
len of vacant_apartments 133
len of new agents 133
year 2027
len of agents to drop 93
len of vacant_apartments 93
len of new agents 93
year 2028
len of agents to drop 112
len of vacant_apartments 112
len of new agents 112
year 2029
len of agents to drop 104
len of vacant_apartments 104
len of new agents 104
year 2030
len of agents to 

In [119]:
df_ts_4.ds[df_ts_4.ds['comment 1']!='']['comment 1'].unique()

array(['death', 'Rental Burden'], dtype=object)

## Condos

In [120]:
def iterate_one_year_SouthTownCondo(y_, work_ds):
    """"""
    # Get list of apartments for year
    se_ = pd.to_datetime(str(y_)+'-1-1')  # start event
    year_subset = work_ds.filter_ds_by_year(
        y_)  # get agents for the current year
    if len(year_subset.ds) == 0:
        print('return year return 0')
        return work_ds
    # update rental prices in working dataset
    year_subset.update_cleanup_agents(cleanupType='Condo')
    # check for agents who died
    dead_agents = work_ds.remove_dead_agents_for_year(se_).copy()
    dead_agents = dead_agents[dead_agents['comment 1'] == '']
    print('Dead Agents:',len(dead_agents))
    dead_agents = ridt_agents_ts_obj.update_cause_of_departure(
        dead_agents, pd.to_datetime(str(y_)+'-5-1'), 'death')  # update case of displaccement
    dead_agents['comment 1'] = 'death'
    year_subset.drop(dead_agents, 'agentID')
    # drop displaced Agents
    # check house prices and update burden and mark potential burden agents
    burden_mark_for_year = mark_burden(year_subset.ds, 0.43, 'Mortgage')
    print('Mortgage Burden Agents:',len(burden_mark_for_year))
    burden_mark_for_year = ridt_agents_ts_obj(burden_mark_for_year)
    m_burden_agents = burden_mark_for_year.get_burden_agents(
        'Mortgage Burden').copy()
    m_burden_agents = ridt_agents_ts_obj.update_cause_of_departure(
        m_burden_agents, pd.to_datetime(str(y_)+'-5-1'), 'Mortgage Burden').copy()  # update case of displaccement
    year_subset.drop(m_burden_agents, 'agentID')  # drop displaced Agents
    print('Mortgage Mortgage Burden:',len(m_burden_agents))

    # ----------------------------------------------------------
    # check Other Accumulated Burdens
    burden_mark_for_year_2 = mark_burden(year_subset.ds, 0.35, 'Total')
    print('Mortgage Total Burden:',len(burden_mark_for_year_2))
    burden_mark_for_year_2 = ridt_agents_ts_obj(burden_mark_for_year_2)

    m_burden_agents_1 = burden_mark_for_year_2.get_burden_agents(
        'Total Burden').copy()
    m_burden_agents_1 = ridt_agents_ts_obj.update_cause_of_departure(
        m_burden_agents_1, pd.to_datetime(str(y_)+'-5-1'), 'Total Burden').copy()  # update case of displaccement
    year_subset.drop(m_burden_agents_1, 'agentID')  # drop displaced Agents
    print('Mortgage Total Burden:',len(m_burden_agents_1))
    # Drop displaced agents from main sets also in future cycles
    # agents to remove
    a2remove = pd.concat([dead_agents, m_burden_agents, m_burden_agents_1])
    if len(a2remove) == 0:
        print('return a2remove is zero')
        return work_ds
    work_ds.drop(year_subset.ds, 'agentID')
    work_ds.drop(dead_agents, 'agentID')
    work_ds.drop(a2remove, 'agentID')  # drop displaced Agents
    print('len of agents to drop', len(a2remove))
    # ----------------------------------------------------------
    # generate new agents in the pool of rental doors
    bld_pool = list(year_subset.ds['bld_key'].unique())
    occupied = list(year_subset.ds['address_door'])
    pool_of_apartments = p_ds.ds[p_ds.ds['bld_key'].isin(bld_pool)].copy()
    pool_of_doors = list(
        pool_of_apartments[pool_of_apartments['ApartmentType'] == 'Condo']['address_door'])
    avialable_doors = [door for door in pool_of_doors if door not in occupied]
    vacant_apartments = pool_of_apartments[pool_of_apartments['address_door'].isin(
        avialable_doors)].copy()
    print('len of vacant_apartments', len(vacant_apartments))

    # --------------------------------------------------------------------------------------------------
    new_agents = ridt_agents_from_bld(
        vacant_apartments, option='copy', year_=y_)
    # --------------------
    new_agents_1 = update_south_town_calc_condo(new_agents.ds, y_)
    # --------------------
    new_agents_1_obj = ridt_agents_obj(new_agents_1)
    new_agents_2 = addPreviousTenantHistory(
        new_agents_1_obj.ds, a2remove)

    print('len of new agents', len(new_agents_2))
    # -------------------------------------------------------------
    work_ds.concat([year_subset.ds, a2remove, new_agents_2])
    # work_ds
    work_ds.ds.reset_index(inplace=True, drop=True)
    return work_ds


In [121]:
def south_town_condo_updater(df_ts, GroupBuildings,last_year):
    query_str = "(Category=='Condo')&(Group=='"+GroupBuildings+"')"
    rental_south_town = p_ds.bldg_dict.query(query_str)
    # get the list of southtown buildings
    rental_bldg_list = list(rental_south_town['bldgs_id'].values)
    # detach all dataset with southtown rental dataset
    main_ds, work_ds = ridt_agents_ts_obj(
        df_ts.ds).detach_by_list_of_bld_keys(rental_bldg_list)
    # make a list of years to modify
    #condos_ds, renters_ds
    work_ds = ridt_agents_ts_obj(work_ds.ds)
    work_ds.update_cleanup_agents('Condo')
    y_=2004
    years_range = range(work_ds.ds['built_year'].min().year,last_year)
    for y_ in years_range:
        print('year',y_)
        work_ds = iterate_one_year_SouthTownCondo(y_,work_ds)
    return(ridt_agents_ts_obj(pd.concat([main_ds.ds,work_ds.ds]).reset_index(drop=True)))

In [122]:
df_ts_5 = south_town_condo_updater(df_ts_4, 'SouthTown',2070)


year 2004
Dead Agents: 0
Mortgage Burden Agents: 230
Mortgage Mortgage Burden: 20
Mortgage Total Burden: 210
Mortgage Total Burden: 180
len of agents to drop 200
len of vacant_apartments 200


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


len of new agents 200
year 2005
Dead Agents: 0
Mortgage Burden Agents: 230
Mortgage Mortgage Burden: 0
Mortgage Total Burden: 230
Mortgage Total Burden: 28
len of agents to drop 28
len of vacant_apartments 28


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


len of new agents 28
year 2006
Dead Agents: 0
Mortgage Burden Agents: 230
Mortgage Mortgage Burden: 0
Mortgage Total Burden: 230
Mortgage Total Burden: 3
len of agents to drop 3
len of vacant_apartments 3
len of new agents 3


...

/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


len of new agents 16
year 2068
Dead Agents: 28
Mortgage Burden Agents: 555
Mortgage Mortgage Burden: 3
Mortgage Total Burden: 552
Mortgage Total Burden: 1
len of agents to drop 32
len of vacant_apartments 32


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


len of new agents 32
year 2069
Dead Agents: 13
Mortgage Burden Agents: 570
Mortgage Mortgage Burden: 2
Mortgage Total Burden: 568
Mortgage Total Burden: 3
len of agents to drop 18
len of vacant_apartments 18
len of new agents 18


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


# Wire 🥳

In [123]:
def replace_dead_agents(agents_ds):
    datr = agents_ds.loc[agents_ds['death_date']<ridt_agents_ts_obj.last_date].copy() # select all the dead agents Simulation Ends
    iteration = 0
    new_agents_accumulator = pd.DataFrame()
    while iteration < 15:
        if (iteration >0):
            conc_ = [new_agents_accumulator,datr]
            new_agents_accumulator = pd.concat(conc_)
            datr = datr.loc[datr['death_date']<ridt_agents_ts_obj.last_date].copy() # select all the dead agents Simulation Ends
            print('Total Dead Agents remaining to replace: '+ str(check))
        check = len( datr.index)
        if check<5:
                iteration = 30
                print('goal Reached')
        else:
            datr = ridt_agents_ts_obj.replace_dead_tenants_once(datr)
        iteration +=1
    print('out of loop')
    new_agents_accumulator = pd.concat([new_agents_accumulator,datr])
    agents_ds = pd.concat([agents_ds,new_agents_accumulator])
    agents_ds.reset_index(inplace=True,drop=True)
    return(agents_ds)


In [124]:
ds_backup = df_ts_5.ds.copy()
df_ts_6 = ridt_agents_ts_obj(ds_backup)
WIRE_Agents = df_ts_6.ds[df_ts_6.ds['Group']=='WIRE']

In [125]:
df_ts_6.drop(WIRE_Agents,'agentID')

In [126]:
WIRE_Agents = replace_dead_agents(WIRE_Agents)

Total Dead Agents remaining to replace: 2151
Total Dead Agents remaining to replace: 1686
Total Dead Agents remaining to replace: 444
Total Dead Agents remaining to replace: 301
Total Dead Agents remaining to replace: 301
Total Dead Agents remaining to replace: 292
Total Dead Agents remaining to replace: 238
Total Dead Agents remaining to replace: 152
Total Dead Agents remaining to replace: 64
Total Dead Agents remaining to replace: 20
goal Reached
out of loop


In [127]:
df_ts_6.concat([WIRE_Agents])

In [128]:
def convert_to_condo(na,start_year) :
    na.ds['affordable_living'] = 0
    na.ds['comment 1'] = 'Partial_Market'
    na.ds['Rental_Units'] = 0
    na.ds['protected'] = False
    na.ds['ApartmentType'] = 'Co-op'
    sqppf = ridt_agent_economy.get_co_op_price_per_feet_based_on_year(start_year.year)
    na.ds['annual_return_mortgage'] = na.ds['Ap_sq_feet_size'].apply(lambda x: ridt_mortgage_calc.calculte_monthly_mortgage(x * sqppf)) * 12
    #na.ds =ridt_agent_economy.modify_annual_expenses_for_co_op(na.ds,start_year)
    get_cofactor = ridt_agent_economy.calculate_hoa_for_condos(start_year)
    na.ds['annual_hoa_expenses'] = get_cofactor * na.ds['Ap_sq_feet_size']
    na.ds['annual_expenses'] =  na.ds['annual_hoa_expenses']+na.ds['annual_return_mortgage']+3000
    min_ = (na.ds['annual_expenses']/0.6).astype(int)
    ### QUESTION FOR SHARON!!!
    max_ = (na.ds['annual_expenses']/0.1).astype(int)
    ap_ds_copy = na.ds.copy()
    ap_ds_copy['min_'] = min_
    ap_ds_copy['max_'] = max_
    na.ds['income'] = ap_ds_copy.apply(lambda x: random.randint(x['min_'],x['max_']),axis=1)

    return na

In [129]:
#def update_Wire_To_Market_rate(p_ds, bldg_events, dfts):
#p_ds,bldg_events,df_ts_6
dfts = ridt_agents_ts_obj(df_ts_6.ds)
"""This function updates the Populatino Time Series according to the WIRE Scenario
Input
-----
p_ds: building object (information about building)
bldg_events: the events of the building
dfts: Population TimeSeries
Output
------
the simulation returns a new dataset with the agents modification according to WIRE Scenario"""
bld_ts = ridt_buildings_ts_obj(
    p_ds.json_bldgs, bldg_events)  # Adds time series to the building DS
# Will need to update for non WIRE eventually
abe = bld_ts.ds[bld_ts.ds['Event'] != "No Events"]
bld_list = list(abe['Bldg Proje'].unique())


In [130]:
#for bld in bld_list:
#cb = bld_list[0]
for cb in bld_list:
    #def updtate_write_to_market_rate_iterate_bld_list(bld_list, abe, dfts, bld_ts,p_ds):
            #for cb in bld_list:
    print('bulding', cb)
    print('')
    fe = abe.loc[(abe['Bldg Proje'] == cb) & (abe['Event'] == 'Flaged')].squeeze()  # fe flaged Event

    #rc = ridt_agents_ts_obj(dfts.filter_ds_by_building(fe['Bldg Proje']).ds)

    bld_key = p_ds.ds[p_ds.ds['Building Name']==cb]['bld_key'].unique()
    if cb == bld_list[0]:
        Other_Buildings, rc = ridt_agents_ts_obj(dfts.ds).detach_by_list_of_bld_keys(bld_key)
    else:
        Other_Buildings, rc = ridt_agents_ts_obj(Other_Buildings.ds).detach_by_list_of_bld_keys(bld_key)
    rc.ds = ridt_WIRE.buy_out_specific_building (rc.ds,fe)
    drop_buy_outs = rc.ds[rc.ds['cause'] == 'Buy Out']
    rc.drop_future_agent_cycles(drop_buy_outs)
    rc.ds = ridt_WIRE.buy_in_specific_building (rc.ds,fe)
    dead_buy_in = rc.ds[rc.ds['comment 2'] == 'Buy IN']
    rc.drop_future_agent_cycles(dead_buy_in)    
    pem = abe.loc[(abe['Bldg Proje'] == cb) & (abe['Event'] == 'Market Rate')].squeeze()  # me market event
    se = pem.start_event  # start event
    ye_ = se.year

    range_ = range(se.year, 2070)
    for ye_ in range_:
        print(ye_)
        print()
        y_data = pd.to_datetime(str(ye_)+'-1-1')
        if ye_==range_[0]:
            rc_year, rc_future = rc.filter_ds_by_year_detach(ye_)
        else:
            rc_year, rc_future = rc_future.filter_ds_by_year_detach(ye_)
        print('only this year:', len(rc_year.ds))
        rc_year.update_cleanup_agents('Rental')
        rc_year.update_cleanup_agents()

        dead_agents = rc_future.remove_dead_agents_for_year(y_data).copy()
        #dead_agents = dead_agents[dead_agents['comment 1'] == '']
        print('Dead Agents:', len(dead_agents))
        dead_agents = ridt_agents_ts_obj.update_cause_of_departure(
            dead_agents, pd.to_datetime(str(ye_)+'-5-1'), 'death')  # update case of displaccement
        dead_agents['comment 1'] = 'death'
        rc_year.drop(dead_agents, 'agentID')
        rc_future.drop_future_agent_cycles(dead_agents)
        #rc_year.drop_future_agent_cycles(dead_agents)
        # drop displaced Agents
        # check house prices and update burden and mark potential burden agents
        burden_mark_for_year = mark_burden(rc_year.ds, 0.43, 'Mortgage')
        burden_mark_for_year = ridt_agents_ts_obj(burden_mark_for_year)
        m_burden_agents = burden_mark_for_year.get_burden_agents(
            'Mortgage Burden').copy()
        m_burden_agents = ridt_agents_ts_obj.update_cause_of_departure(
            m_burden_agents, pd.to_datetime(str(ye_)+'-5-1'), 'Mortgage Burden').copy()  # update case of displaccement
        rc_year.drop(m_burden_agents, 'agentID')  # drop displaced Agents
        print('Mortgage Mortgage Burden:', len(m_burden_agents))
        rc_future.drop_future_agent_cycles(m_burden_agents)
        #rc_year.drop_future_agent_cycles(m_burden_agents)
        # ----------------------------------------------------------
        # check Other Accumulated Burdens
        burden_mark_for_year_2 = mark_burden(rc_year.ds, 0.35, 'Total')
        print('Total Burden:', len(burden_mark_for_year_2))
        burden_mark_for_year_2 = ridt_agents_ts_obj(burden_mark_for_year_2)

        m_burden_agents_1 = burden_mark_for_year_2.get_burden_agents(
            'Total Burden').copy()
        m_burden_agents_1 = ridt_agents_ts_obj.update_cause_of_departure(
            m_burden_agents_1, pd.to_datetime(str(ye_)+'-5-1'), 'Total Burden').copy()  # update case of displaccement
        rc_year.drop(m_burden_agents_1, 'agentID')  # drop displaced Agents
        print('Total Burden:', len(m_burden_agents_1))
        rc_future.drop_future_agent_cycles(m_burden_agents_1)
        #rc_year.drop_future_agent_cycles(m_burden_agents_1)
        a2remove = pd.concat([dead_agents, m_burden_agents, m_burden_agents_1])
        print('total a2remove:',len(a2remove))
        if len(a2remove) == 0:
            print('return a2remove is zero')
            #return work_ds
        else:
            rc_future.drop(a2remove,'agentID')
            # ----------------------------------------------------------
        # generate new agents in the pool of rental doors
        bld_pool = list(rc_year.ds['bld_key'].unique())
        occupied = list(rc_year.ds['address_door'])
        pool_of_apartments = p_ds.ds[p_ds.ds['bld_key'].isin(bld_pool)].copy()
        pool_of_doors = list(pool_of_apartments['address_door'])
        avialable_doors = [door for door in pool_of_doors if door not in occupied]
        vacant_apartments = pool_of_apartments[pool_of_apartments['address_door'].isin(
            avialable_doors)].copy()
        print('len of vacant_apartments', len(vacant_apartments))
        if len(vacant_apartments)>0:
            new_agents = ridt_agents_from_bld(
                vacant_apartments, option='copy', year_=ye_)
            new_agents = convert_to_condo(new_agents,y_data)
            new_agents.update_cleanup_agents()
            new_agents_2 = addPreviousTenantHistory(
                    new_agents.ds, a2remove)
            this_year_updated = pd.concat([rc_year.ds, a2remove, new_agents_2])
            # year_name=str(ye_)+'_.xlsx'
            # this_year_updated.to_excel('scratch/years_1/'+year_name)
            # -------------------------------------------------------------
            rc_future.concat([this_year_updated])
            # work_ds
        else:
            rc_future.concat([rc_year.ds])
        rc_future.ds.reset_index(inplace=True, drop=True)
    Other_Buildings.concat([rc_future.ds])
    Other_Buildings.ds.reset_index(inplace=True, drop=True)

df_ts_10 = ridt_agents_ts_obj(Other_Buildings.ds)


bulding Rivercross

2014

only this year: 366
Dead Agents: 8
Mortgage Mortgage Burden: 0
Total Burden: 366
Total Burden: 2
total a2remove: 10
len of vacant_apartments 13


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


2015

only this year: 372
Dead Agents: 5
Mortgage Mortgage Burden: 1
Total Burden: 371
Total Burden: 0
total a2remove: 6
len of vacant_apartments 6


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


2016

only this year: 370
Dead Agents: 7
Mortgage Mortgage Burden: 1
Total Burden: 369
Total Burden: 0
total a2remove: 8
len of vacant_apartments 8


...

/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


2069

only this year: 369
Dead Agents: 2
Mortgage Mortgage Burden: 0
Total Burden: 369
Total Burden: 1
total a2remove: 3
len of vacant_apartments 3


/Users/shai/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


bulding Roosevelt Landings



IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
#df_ts_10.ds.to_excel('scratch/test_1459.xlsx')
df_ts_10.ds.to_excel('result_july_2_2021_1839.xlsx')